# H3-H4 dimer with truncated tails in water
## Minimize and equilibrate
 - Minimize
 - Equilibrate
 
 Sanity checks along the way.
 
 Protocol files are copied from https://github.com/intbio/gmx_protocols/tree/master/amber into
Protocols folder.
If others are needed modify then in the folder.

### Software packages
- Python 3
- `conda install -c intbio gromacs=2018.3`
- `conda install -c conda-forge jupyterlab` OR `conda install jupyter`
- `conda install -c conda-forge mdanalysis`
- `conda install -c conda-forge wget`
- `conda install nglview -c conda-forge`

For jupyterlab
- `conda install nodejs`
- `jupyter-labextension install @jupyter-widgets/jupyterlab-manager@0.33.2`
- `jupyter-labextension install nglview-js-widgets@1.1.2`

In [19]:
#Libraries and main variables initialized
import MDAnalysis as mda
from MDAnalysis.analysis import align
from MDAnalysis.analysis.rms import rmsd
import numpy as np
import urllib.request, json
import os
import re
import nglview as nv
from funcs import get_files_from_git
from funcs import view_nucl



%matplotlib inline
folder="h3-h4_tm" #This one is important - set it to a unique value

In [ ]:
%%bash
rm -rf MDProtocols

In [ ]:
#Set protocols URL
prot_url="https://api.github.com/repos/intbio/gmx_protocols/contents/amber"

In [ ]:
get_files_from_git(prot_url,'MDProtocols/')

### 1_Minimization - steepest descent gradient method 10000 steps with POSRED constraints

#### Prepare tpr files using GROMPP
- http://manual.gromacs.org/documentation/current/onlinehelp/gmx-grompp.html

In [ ]:
%%bash
#Prepare system
mkdir -p GMX_run
gmx grompp -f MDProtocols/1_minim.mdp -c GMX_system/init_solv_ions.pdb -p GMX_system/topol.top \
-o GMX_run/1_minim.tpr -r GMX_system/init_solv_ions.pdb -n GMX_system/index.ndx -po GMX_run/mdout_1_minim.mdp


#### Run: Option 1 - locally

In [ ]:
%%bash
cd GMX_run
gmx mdrun -nt 20 -deffnm 1_minim # if you want to run locally, nt is the number of threads

#### Run: Option 2 - sending and simulating on Lomonosov-2

In [35]:
%%bash -s "$folder" --err err
#Make folder remotely
#echo "Executing:"
#echo "ssh lomo2 \"mkdir -p ~/_scratch/$1\""
ssh lomo2 "mkdir -p ~/_scratch/$1" 2>/dev/null


In [ ]:
%%bash -s "$folder" --err err --out out
#Copy to SC

cp Run_scripts/lomo2_run.sh GMX_run/
scp -r GMX_run lomo2:~/_scratch/$1/;

In [ ]:
%%bash -s "$folder" 
#--out out --err err
ssh lomo2 "module load slurm gromacs/2018-gcc; cd ~/_scratch/$1/GMX_run; \
sbatch -p test -t 00:15:00 -N 2 -J $1 lomo2_run.sh 1_minim"
#gmx mdrun -deffnm GMX_run/minimization # if you want to run locally

In [ ]:
#get job id
print(out)
job_idm=re.findall(r'\d+', out)[0]
print(job_idm)

In [ ]:
%%bash -s "$job_idm"   --err err
# check job status
ssh lomo2 "module load slurm gromacs/2018-gcc; squeue -j $1 " || echo "No job in queue"

In [ ]:
%%bash -s "$folder"  "$job_idm"  --err err
# check job status
ssh lomo2 "tail -n 25 ~/_scratch/$1/GMX_run/egmx.$2"

In [ ]:
%%bash -s "$folder" --err err 
#Purge # files

ssh lomo2 "rm -f ~/_scratch/$1/GMX_run/\#*; ls ~/_scratch/$1/GMX_run/"

In [ ]:
%%bash -s "$folder" --err err --out out
#Get back data

scp lomo2:~/_scratch/$1/GMX_run/* GMX_run/ ;

#### Analyze what we get

In [18]:
view_nucl('GMX_system/init_solv_ions.pdb','GMX_run/1_minim.trr')

NGLWidget(count=7)

### 2_Equilibration 200 ps with postional restraints of 500 kJ/mol/A2

In [ ]:
%%bash  
#--out out --err err
gmx grompp -f MDProtocols/2_equil.mdp -c GMX_run/1_minim.gro -p GMX_system/topol.top \
-o GMX_run/2_equil.tpr -v  -r GMX_run/1_minim.gro -n GMX_system/index.ndx -po GMX_run/mdout_2_equil.mdp


#### Run: Option 1 - locally

In [ ]:
%%bash
cd GMX_run
gmx mdrun -deffnm 2_equil # if you want to run locally

#### Run: Option 2 - sending and simulating on Lomonosov-2

In [ ]:
%%bash -s "$folder" --err err --out out
#Copy to SC

cp Run_scripts/lomo2_run.sh GMX_run/
rsync -r GMX_run lomo2:~/_scratch/$1/;

In [ ]:
%%bash -s "$folder" --out out --err err
ssh lomo2 "module load slurm gromacs/2018-gcc; cd ~/_scratch/$1/GMX_run; \
sbatch -p compute -t 00:15:00 -N 5 lomo2_run.sh 2_equil"

In [ ]:
#get job id
print(out)
job_ide=re.findall(r'\d+', out)[0]
print(job_ide)

In [ ]:
#job_ide=869439

In [ ]:
%%bash -s   "$job_ide"   --err err
# check job status
ssh lomo2 "module load slurm gromacs/2018-gcc; squeue -j $1 " || echo "No job in queue"

In [ ]:
%%bash -s "$folder"  "$job_ide"  --err err
# check job status
ssh lomo2 "tail -n 10 ~/_scratch/$1/GMX_run/egmx.$2"

In [ ]:
%%bash -s "$folder" --err err 
#Purge # files

ssh lomo2 "rm -f ~/_scratch/$1/GMX_run/\#*; ls ~/_scratch/$1/GMX_run/"

In [ ]:
%%bash -s "$folder" --err err --out out
#Get back data

rsync --partial lomo2:~/_scratch/$1/GMX_run/2_equil* GMX_run/ ;

#### Analyze what we get

In [ ]:
%%bash --bg
cd GMX_run
vmd ../GMX_system/init_solv_ions.pdb 2_equil.xtc

### 3_Equilibration 200 ps with postional restraints of 50 kJ/mol/A2

In [ ]:
%%bash  
#--out out --err err
gmx grompp -f MDProtocols/2_equil.mdp -c GMX_run/2_equil.gro -p GMX_system/topol.top \
-o GMX_run/3_equil.tpr -v  -r GMX_run/2_equil.gro -n GMX_system/index.ndx -po GMX_run/mdout_3_equil.mdp


#### Run: Option 1 - locally

In [ ]:
%%bash
cd GMX_run
gmx mdrun -deffnm 3_equil # if you want to run locally

#### Run: Option 2 - sending and simulating on Lomonosov-2

In [ ]:
%%bash -s "$folder" --err err --out out
#Copy to SC

cp Run_scripts/lomo2_run.sh GMX_run/
rsync -r GMX_run lomo2:~/_scratch/$1/;

In [ ]:
%%bash -s "$folder" --out out --err err
ssh lomo2 "module load slurm gromacs/2018-gcc; cd ~/_scratch/$1/GMX_run; \
sbatch -p compute -t 00:15:00 -N 5 lomo2_run.sh 3_equil"

In [ ]:
#get job id
print(out)
job_ide=re.findall(r'\d+', out)[0]
print(job_ide)

In [ ]:
#job_ide=869439

In [ ]:
%%bash -s   "$job_ide"   --err err
# check job status
ssh lomo2 "module load slurm gromacs/2018-gcc; squeue -j $1 " || echo "No job in queue"

In [ ]:
%%bash -s "$folder"  "$job_ide"  --err err
# check job status
ssh lomo2 "tail -n 10 ~/_scratch/$1/GMX_run/egmx.$2"

In [ ]:
%%bash -s "$folder" --err err 
#Purge # files

ssh lomo2 "rm -f ~/_scratch/$1/GMX_run/\#*; ls ~/_scratch/$1/GMX_run/"

In [ ]:
%%bash -s "$folder" --err err --out out
#Get back data

rsync --partial lomo2:~/_scratch/$1/GMX_run/3_equil* GMX_run/ ;

#### Analyze what we get

In [ ]:
%%bash --bg
cd GMX_run
vmd ../GMX_system/init_solv_ions.pdb 3_equil.xtc

### 4_Equilibration 200 ps with postional restraints of 5 kJ/mol/A2

In [ ]:
%%bash  
#--out out --err err
gmx grompp -f Protocols/3_equil.mdp -c GMX_run/3_equil.gro -p GMX_system/topol.top \
-o GMX_run/4_equil.tpr -v  -r GMX_run/3_equil.gro -n GMX_system/index.ndx -po GMX_run/mdout_4_equil.mdp


#### Run: Option 1 - locally

In [ ]:
%%bash
cd GMX_run
gmx mdrun -deffnm 4_equil # if you want to run locally

#### Run: Option 2 - sending and simulating on Lomonosov-2

In [ ]:
%%bash -s "$folder" --err err --out out
#Copy to SC

cp Run_scripts/lomo2_run.sh GMX_run/
rsync -r GMX_run lomo2:~/_scratch/$1/;

In [ ]:
%%bash -s "$folder" --out out --err err
ssh lomo2 "module load slurm gromacs/2018-gcc; cd ~/_scratch/$1/GMX_run; \
sbatch -p compute -t 00:15:00 -N 5 lomo2_run.sh 4_equil"

In [ ]:
#get job id
print(out)
job_ide=re.findall(r'\d+', out)[0]
print(job_ide)

In [ ]:
job_ide=869453

In [ ]:
%%bash -s   "$job_ide"   --err err
# check job status
ssh lomo2 "module load slurm gromacs/2018-gcc; squeue -j $1 " || echo "No job in queue"

In [ ]:
%%bash -s "$folder"  "$job_ide"  --err err
# check job status
ssh lomo2 "tail -n 10 ~/_scratch/$1/GMX_run/egmx.$2"

In [ ]:
%%bash -s "$folder" --err err 
#Purge # files

ssh lomo2 "rm -f ~/_scratch/$1/GMX_run/\#*; ls ~/_scratch/$1/GMX_run/"

In [ ]:
%%bash -s "$folder" --err err --out out
#Get back data

rsync --partial lomo2:~/_scratch/$1/GMX_run/4_equil* GMX_run/ ;
#scp  lomo2:~/_scratch/$1/GMX_run/4_equil* GMX_run/ ; #sometimes rsync does not work(why?)

#### Analyze what we get

In [ ]:
%%bash --bg
cd GMX_run
vmd ../GMX_system/init_solv_ions.pdb 4_equil.xtc

### 5_Equilibration 200 ps with postional restraints of 0.5 kJ/mol/A2

In [ ]:
%%bash  
#--out out --err err
gmx grompp -f MDProtocols/4_equil.mdp -c GMX_run/4_equil.gro -p GMX_system/topol.top \
-o GMX_run/5_equil.tpr -v  -r GMX_run/4_equil.gro -n GMX_system/index.ndx -po GMX_run/mdout_5_equil.mdp


#### Run: Option 1 - locally

In [ ]:
%%bash
cd GMX_run
gmx mdrun -deffnm 5_equil # if you want to run locally

#### Run: Option 2 - sending and simulating on Lomonosov-2

In [ ]:
%%bash -s "$folder" --err err --out out
#Copy to SC

cp Run_scripts/lomo2_run.sh GMX_run/
rsync -r GMX_run lomo2:~/_scratch/$1/;

In [ ]:
%%bash -s "$folder" --out out --err err
ssh lomo2 "module load slurm gromacs/2018-gcc; cd ~/_scratch/$1/GMX_run; \
sbatch -p compute -t 00:15:00 -N 5 lomo2_run.sh 5_equil"

In [ ]:
#get job id
print(out)
job_ide=re.findall(r'\d+', out)[0]
print(job_ide)

In [ ]:
#job_ide=869439

In [ ]:
%%bash -s   "$job_ide"   --err err
# check job status
ssh lomo2 "module load slurm gromacs/2018-gcc; squeue -j $1 " || echo "No job in queue"

In [ ]:
%%bash -s "$folder"  "$job_ide"  --err err
# check job status
ssh lomo2 "tail -n 10 ~/_scratch/$1/GMX_run/egmx.$2"

In [ ]:
%%bash -s "$folder" --err err 
#Purge # files

ssh lomo2 "rm -f ~/_scratch/$1/GMX_run/\#*; ls ~/_scratch/$1/GMX_run/"

In [ ]:
%%bash -s "$folder" --err err --out out
#Get back data

rsync --partial lomo2:~/_scratch/$1/GMX_run/5_equil* GMX_run/ ;

#### Analyze what we get

In [ ]:
%%bash --bg
cd GMX_run
vmd ../GMX_system/init_solv_ions.pdb 5_equil.xtc

### 6_Equilibration 200 ps unrestrained

In [ ]:
%%bash  
#--out out --err err
gmx grompp -f MDProtocols/5_equil.mdp -c GMX_run/5_equil.gro -p GMX_system/topol.top \
-o GMX_run/6_equil.tpr -v  -r GMX_run/5_equil.gro -n GMX_system/index.ndx -po GMX_run/mdout_6_equil.mdp


#### Run: Option 1 - locally

In [ ]:
%%bash
cd GMX_run
gmx mdrun -deffnm 6_equil # if you want to run locally

#### Run: Option 2 - sending and simulating on Lomonosov-2

In [ ]:
%%bash -s "$folder" --err err --out out
#Copy to SC

cp Run_scripts/lomo2_run.sh GMX_run/
rsync -r GMX_run lomo2:~/_scratch/$1/;

In [ ]:
%%bash -s "$folder" --out out --err err
ssh lomo2 "module load slurm gromacs/2018-gcc; cd ~/_scratch/$1/GMX_run; \
sbatch -p compute -t 00:15:00 -N 5 lomo2_run.sh 6_equil"

In [ ]:
#get job id
print(out)
job_ide=re.findall(r'\d+', out)[0]
print(job_ide)

In [ ]:
#job_ide=869439

In [ ]:
%%bash -s   "$job_ide"   --err err
# check job status
ssh lomo2 "module load slurm gromacs/2018-gcc; squeue -j $1 " || echo "No job in queue"

In [ ]:
%%bash -s "$folder"  "$job_ide"  --err err
# check job status
ssh lomo2 "tail -n 10 ~/_scratch/$1/GMX_run/egmx.$2"

In [ ]:
%%bash -s "$folder" --err err 
#Purge # files

ssh lomo2 "rm -f ~/_scratch/$1/GMX_run/\#*; ls ~/_scratch/$1/GMX_run/"

In [ ]:
%%bash -s "$folder" --err err --out out
#Get back data

rsync --partial lomo2:~/_scratch/$1/GMX_run/6_equil* GMX_run/ ;

#### Analyze what we get

In [ ]:
%%bash --bg
cd GMX_run
vmd ../GMX_system/init_solv_ions.pdb 6_equil.xtc

As a result we have equlibrated system ready for production simulations.
We copy it to GMX_system.

In [ ]:
%%bash
cp GMX_run/6_equil.gro GMX_system/

#following files will be needed
#gmx grompp -f Protocols/7_prod.mdp -c GMX_system/6_equil.gro -p GMX_system/topol.top \
#-o GMX_run/7_prod.tpr -v  -r GMX_system/6_equil.gro -n GMX_system/index.ndx -po GMX_run/mdout_7_prod.mdp

# Producing final results

The main aim of this repo is to produce equilibrated simulation systems, here we place into results folder the set of equilibrated files of step 6.


In [ ]:
%%bash
gmx trjconv -f GMX_run/6_equil.gro -s GMX_run/6_equil.tpr -o GMX_run/6_equil.pdb << !
0
!

In [ ]:
%%bash --out out
gmx trjconv -f GMX_system/init_solv_ions.pdb -s GMX_run/6_equil.tpr -o results/init_prot_xray.pdb << !
1
!


In [ ]:
%%bash
mkdir -p results
cp GMX_run/6_equil* results/